In [1]:
corpus_list = ['BibleTTS/akuapem-twi',
    'BibleTTS/asante-twi',
    'BibleTTS/ewe',
    'BibleTTS/hausa',
    'BibleTTS/lingala'
    'BibleTTS/yoruba',
    'Buckeye',
    'EUROM',
    'LibriSpeech',
    'LibriVox',
    'MediaSpeech/AR',
    'MediaSpeech/ES',
    'MediaSpeech/FR',
    'MediaSpeech/TR',
    'MozillaCommonVoice/ab',
    'MozillaCommonVoice/ar',
    'MozillaCommonVoice/ba',
    'MozillaCommonVoice/be',
    'MozillaCommonVoice/bg',
    'MozillaCommonVoice/bn',
    'MozillaCommonVoice/br',
    'MozillaCommonVoice/ca',
    'MozillaCommonVoice/ckb',
    'MozillaCommonVoice/cnh',
    'MozillaCommonVoice/cs',
    'MozillaCommonVoice/cv',
    'MozillaCommonVoice/cy',
    'MozillaCommonVoice/da',
    'MozillaCommonVoice/de',
    'MozillaCommonVoice/dv',
    'MozillaCommonVoice/el',
    'MozillaCommonVoice/en',
    'MozillaCommonVoice/eo',
    'MozillaCommonVoice/es',
    'MozillaCommonVoice/et',
    'MozillaCommonVoice/eu',
    'MozillaCommonVoice/fa',
    'MozillaCommonVoice/fi',
    'MozillaCommonVoice/fr',
    'MozillaCommonVoice/fy-NL',
    'MozillaCommonVoice/ga-IE',
    'MozillaCommonVoice/gl',
    'MozillaCommonVoice/gn',
    'MozillaCommonVoice/hi',
    'MozillaCommonVoice/hu',
    'MozillaCommonVoice/hy-AM',
    'MozillaCommonVoice/id',
    'MozillaCommonVoice/ig',
    'MozillaCommonVoice/it',
    'MozillaCommonVoice/ja',
    'MozillaCommonVoice/ka',
    'MozillaCommonVoice/kab',
    'MozillaCommonVoice/kk',
    'MozillaCommonVoice/kmr',
    'MozillaCommonVoice/ky',
    'MozillaCommonVoice/lg',
    'MozillaCommonVoice/lt',
    'MozillaCommonVoice/ltg',
    'MozillaCommonVoice/lv',
    'MozillaCommonVoice/mhr',
    'MozillaCommonVoice/ml',
    'MozillaCommonVoice/mn',
    'MozillaCommonVoice/mt',
    'MozillaCommonVoice/nan-tw',
    'MozillaCommonVoice/nl',
    'MozillaCommonVoice/oc',
    'MozillaCommonVoice/or',
    'MozillaCommonVoice/pl',
    'MozillaCommonVoice/pt',
    'MozillaCommonVoice/ro',
    'MozillaCommonVoice/ru',
    'MozillaCommonVoice/rw',
    'MozillaCommonVoice/sr',
    'MozillaCommonVoice/sv-SE',
    'MozillaCommonVoice/sw',
    'MozillaCommonVoice/ta',
    'MozillaCommonVoice/th',
    'MozillaCommonVoice/tr',
    'MozillaCommonVoice/tt',
    'MozillaCommonVoice/ug',
    'MozillaCommonVoice/uk',
    'MozillaCommonVoice/ur',
    'MozillaCommonVoice/uz',
    'MozillaCommonVoice/vi',
    'MozillaCommonVoice/yo',
    'MozillaCommonVoice/yue',
    'MozillaCommonVoice/zh-CN',
    'MozillaCommonVoice/zh-TW',
    'primewords_chinese',
    'room_reader',
    'SpeechClarity',
    'TAT-Vol2',
    'thchs30',
    'TIMIT',
    'TTS_Javanese',
    'zeroth_korean',]

In [7]:
import scipy.io as sio
import pandas as pd
import numpy as np
import glob
import csv

def make_meta_file(corpus):
    
    params_list = glob.glob('STM_output/Survey/speech_params_'+corpus+'/*')
    df_list = []

    # load the data from the mat file
    for params_file in params_list:
        data_dict = sio.loadmat(params_file)
        structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
        df = pd.DataFrame(structure_dict)
        df.drop(columns=['x_axis','y_axis'], inplace=True)
        df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        df_list.append(df)
    
    df_all = pd.concat(df_list, ignore_index=True)

    # add the speaker ID and gender info
    if 'MozillaCommonVoice' in corpus:
        valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)
        valid_df['path'] = valid_df['path'].str.replace('.mp3', '')
        valid_df.rename(columns={'client_id':'speaker/player'}, inplace=True)
        df_all = df_all.merge(valid_df[['speaker/player', 'path', 'gender']], how='left', left_on='filename', right_on='path').drop(columns=['path'])
    elif 'BibleTTS' in corpus:
        df_all['speaker/player'] = 'BibleTTS_' + df_all['filename'].str[:3]
        df_all['gender'] = np.nan
    elif 'Buckeye' in corpus:
        df_all['speaker/player'] = df_all['filename'].str[:3].str.replace('s', 'S')
        df_gender = pd.read_csv('data/speechCorp/Buckeye/Buckeye_speaker_info.csv')
        df_all = df_all.merge(df_gender[['SPEAKER', "SPEAKER'S GENDER"]], how='left', left_on='speaker/player', right_on='SPEAKER').drop(columns=['SPEAKER'])
        df_all.rename(columns={"SPEAKER'S GENDER":'gender'}, inplace=True)
        df_all['speaker/player'] = 'Buckeye_'+df_all['speaker/player']


    return df_all

In [16]:
corpus = 'MozillaCommonVoice/hi'
df_all = make_meta_file(corpus)
df_all

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,speaker/player,gender
0,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_26986130,Hindi,1,1,128000,32,4,0.903469,4,d24af5ee15190d3fc14dfc4ee3c75c75fe66fb537d4c7c...,male
1,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_26021896,Hindi,1,1,128000,32,4,0.666188,4,0f018a99663f33afbb7d38aee281fb1afcfd07f9e7acd0...,NaN
2,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_24026258,Hindi,1,1,128000,32,4,0.668438,4,63293ff4cc67280b10da2df2fd2763db65ca8493af191e...,male
3,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_25170240,Hindi,1,1,128000,32,4,0.620031,4,c88824395561074e0eb0ebc1280d4866dbc29f01c02689...,NaN
4,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_23849297,Hindi,1,1,192000,48,4,0.792937,4,46f23ae11e8565e979749342e1097a77fe3ef574253f10...,male
...,...,...,...,...,...,...,...,...,...,...,...,...
5433,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_27761274,Hindi,1,1,128000,32,4,0.164656,4,d5880664e0b201cf24b4972ea9890b801afe05af05cd43...,male
5434,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_25241159,Hindi,1,1,128000,32,4,0.803937,4,3255dc44d5426ad255e14c4db4d12b224dfffd1e594a40...,male
5435,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_27517919,Hindi,1,1,128000,32,4,0.212375,4,d5880664e0b201cf24b4972ea9890b801afe05af05cd43...,male
5436,data/speechCorp/MozillaCommonVoice/hi/clips/co...,common_voice_hi_32264979,Hindi,1,1,256000,32,8,0.392625,8,b0900acc53b0b578a071411f8ec0479ebf48be094bdcae...,NaN


## test code ##

In [34]:
corpus = 'Buckeye'
params_list = glob.glob('STM_output/Survey/speech_params_'+corpus+'/*')
df_list = []

for params_file in params_list:
    data_dict = sio.loadmat(params_file)
    structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
    df = pd.DataFrame(structure_dict)
    df.drop(columns=['x_axis','y_axis'], inplace=True)
    df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    df_list.append(df)

df_all = pd.concat(df_list, ignore_index=True)
df_all.head()

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur
0,data/speechCorp/Buckeye/s30/s3003a/s3003a.wav,s3003a,English-US,1,1,1920000,16,120,0.983750,120
1,data/speechCorp/Buckeye/s26/s2603b/s2603b.wav,s2603b,English-US,1,1,2048000,16,128,0.935063,120
2,data/speechCorp/Buckeye/s27/s2703a/s2703a.wav,s2703a,English-US,1,1,3328000,16,208,0.963437,120
3,data/speechCorp/Buckeye/s09/s0903a/s0903a.wav,s0903a,English-US,1,1,1920000,16,120,0.399062,120
4,data/speechCorp/Buckeye/s08/s0803b/s0803b.wav,s0803b,English-US,1,1,1984000,16,124,0.978313,120


In [35]:
df_all['speaker/player'] = df_all['filename'].str[:3].str.replace('s', 'S')
df_gender = pd.read_csv('data/speechCorp/Buckeye/Buckeye_speaker_info.csv')

df_all = df_all.merge(df_gender[['SPEAKER', "SPEAKER'S GENDER"]], how='left', left_on='speaker/player', right_on='SPEAKER').drop(columns=['SPEAKER'])
df_all.rename(columns={"SPEAKER'S GENDER":'gender'}, inplace=True)
df_all['speaker/player'] = 'Buckeye_'+df_all['speaker/player']

df_all

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,speaker/player,gender
0,data/speechCorp/Buckeye/s30/s3003a/s3003a.wav,s3003a,English-US,1,1,1920000,16,120,0.983750,120,Buckeye_S30,m
1,data/speechCorp/Buckeye/s26/s2603b/s2603b.wav,s2603b,English-US,1,1,2048000,16,128,0.935063,120,Buckeye_S26,f
2,data/speechCorp/Buckeye/s27/s2703a/s2703a.wav,s2703a,English-US,1,1,3328000,16,208,0.963437,120,Buckeye_S27,f
3,data/speechCorp/Buckeye/s09/s0903a/s0903a.wav,s0903a,English-US,1,1,1920000,16,120,0.399062,120,Buckeye_S09,f
4,data/speechCorp/Buckeye/s08/s0803b/s0803b.wav,s0803b,English-US,1,1,1984000,16,124,0.978313,120,Buckeye_S08,f
...,...,...,...,...,...,...,...,...,...,...,...,...
250,data/speechCorp/Buckeye/s05/s0502b/s0502b.wav,s0502b,English-US,1,1,1920000,16,120,0.738563,120,Buckeye_S05,f
251,data/speechCorp/Buckeye/s37/s3701a/s3701a.wav,s3701a,English-US,1,1,2560000,16,160,0.955438,120,Buckeye_S37,f
252,data/speechCorp/Buckeye/s21/s2101b/s2101b.wav,s2101b,English-US,1,1,1920000,16,120,0.888813,120,Buckeye_S21,f
253,data/speechCorp/Buckeye/s13/s1302a/s1302a.wav,s1302a,English-US,1,1,1984000,16,124,0.959250,120,Buckeye_S13,m
